In [1]:
import pandas as pd
import numpy as np

In [2]:
webExtract = pd.read_csv("data/nyc_inspection_data/WebExtract.txt").fillna(0)

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
webExtract.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462.0,7188924968,8,2014-03-03 00:00:00,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225.0,7182875005,39,2014-07-01 00:00:00,F,06A,23.0,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22 00:00:00,D,10B,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2014-05-29 00:00:00,D,08C,10.0,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22 00:00:00,D,02G,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [4]:
webExtract["INSPDATE"] = pd.to_datetime(webExtract.INSPDATE)

In [5]:
#webExtract = webExtract[webExtract.INSPDATE >= pd.to_datetime("2014-01-01 00:00:00")]

In [6]:
#webExtract = webExtract.sort_values(by=["INSPDATE","SCORE"])
#webExtract = webExtract.groupby("CAMIS").last()

In [7]:
#webExtract = webExtract.groupby(['CUISINECODE','VIOLCODE']).filter(lambda x: len(x)>100)

In [8]:
conditionalgroup = webExtract.groupby('CUISINECODE').VIOLCODE.apply(lambda g: g.value_counts()/len(g)).to_frame()

In [9]:
webExtract.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINECODE,INSPDATE,ACTION,VIOLCODE,SCORE,CURRENTGRADE,GRADEDATE,RECORDDATE
0,30075445,MORRIS PARK BAKE SHOP,2,1007,MORRIS PARK AVE ...,10462.0,7188924968,8,2014-03-03,D,10F,2.0,A,2014-03-03 00:00:00,2014-09-04 06:01:28.403000000
1,30112340,WENDY'S,3,469,FLATBUSH AVENUE,11225.0,7182875005,39,2014-07-01,F,06A,23.0,B,2014-07-01 00:00:00,2014-09-04 06:01:28.403000000
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22,D,10B,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000
3,40356483,WILKEN'S FINE FOOD,3,7114,AVENUE U,11234.0,7184443838,27,2014-05-29,D,08C,10.0,A,2014-05-29 00:00:00,2014-09-04 06:01:28.403000000
4,30191841,DJ REYNOLDS PUB AND RESTAURANT,1,351,WEST 57 STREET,10019.0,2122452912,3,2013-07-22,D,02G,11.0,A,2013-07-22 00:00:00,2014-09-04 06:01:28.403000000


In [10]:
conditionaldf = conditionalgroup.reset_index()

In [11]:
conditionaldf.columns = ["CUISINECODE","VIOLCODE","CP"]

In [12]:
conditionaldf.head()

,CUISINECODE,VIOLCODE,CP
0,0,04A,0.115108
1,0,10F,0.107914
2,0,10B,0.071942
3,0,0,0.057554
4,0,04L,0.057554


In [13]:
unconditionaldf = webExtract.groupby("VIOLCODE")['VIOLCODE'].apply(lambda x: x.count()/float(len(webExtract))).to_frame()

In [14]:
unconditionaldf.columns = ["UNCP"]

In [15]:
unconditionaldf = unconditionaldf.reset_index()

In [16]:
unconditionaldf.head()

,VIOLCODE,UNCP
0,0,0.021083
1,02A,0.000936
2,02B,0.046101
3,02C,0.001034
4,02D,0.000098


In [17]:
pmerge = pd.merge(conditionaldf,unconditionaldf,on="VIOLCODE")

In [18]:
pmerge["ratio"] = pmerge.CP/pmerge.UNCP

In [19]:
pmerge.head()

,CUISINECODE,VIOLCODE,CP,UNCP,ratio
0,0,04A,0.115108,0.017416,6.609448
1,1,04A,0.019544,0.017416,1.122207
2,2,04A,0.045818,0.017416,2.630839
3,3,04A,0.017099,0.017416,0.981803
4,4,04A,0.033537,0.017416,1.925657


In [20]:
pmergesort = pmerge.sort_values(by="ratio").groupby("CUISINECODE").last().reset_index()

In [21]:
pmergesort = pmerge
pmergesort.head()

,CUISINECODE,VIOLCODE,CP,UNCP,ratio
0,0,04A,0.115108,0.017416,6.609448
1,1,04A,0.019544,0.017416,1.122207
2,2,04A,0.045818,0.017416,2.630839
3,3,04A,0.017099,0.017416,0.981803
4,4,04A,0.033537,0.017416,1.925657


In [22]:
pmergesort.ratio[0:20]

0     6.609448
1     1.122207
2     2.630839
3     0.981803
4     1.925657
5     0.878494
6     0.452123
7     0.708884
8     0.854552
9     1.340260
10    0.562937
11    3.435360
12    0.592828
13    1.786471
14    0.926122
15    1.361300
16    1.635606
17    0.489636
18    0.648077
19    0.434668
Name: ratio, dtype: float64

In [23]:
numberdf = webExtract.groupby(['CUISINECODE','VIOLCODE']).VIOLCODE.count().to_frame()

In [24]:
numberdf.columns = ["number"]

In [25]:
numberdf = numberdf.reset_index()

In [26]:
numberdf.head()

,CUISINECODE,VIOLCODE,number
0,0,0,8
1,0,02B,1
2,0,02G,3
3,0,04A,16
4,0,04C,1


In [27]:
result = pd.merge(pmergesort,numberdf,on=["CUISINECODE","VIOLCODE"])

In [28]:
result[result.CUISINECODE==14]

,CUISINECODE,VIOLCODE,CP,UNCP,ratio,number
13,14,04A,0.031113,0.017416,1.786471,474
95,14,10F,0.171316,0.124865,1.372011,2610
178,14,10B,0.061503,0.058900,1.044195,937
261,14,0,0.024614,0.021083,1.167476,375
341,14,04L,0.060584,0.072013,0.841300,923
422,14,08A,0.080735,0.093177,0.866473,1230
503,14,05D,0.014769,0.011204,1.318112,225
578,14,10H,0.023761,0.016406,1.448304,362
661,14,06D,0.059928,0.058214,1.029442,913
742,14,15L,0.003807,0.005880,0.647407,58


In [29]:
result.sort_values(by="ratio").ratio[::-1][0:20].mean()

73.712375989227397

In [30]:
cuisine = pd.read_csv("data/nyc_inspection_data/Cuisine.txt")

In [31]:
cuisine.head()

,CUISINECODE,CODEDESC
0,2,African
1,3,American
2,5,Asian
3,15,Cajun
4,17,Caribbean


In [32]:
merge1 = pd.merge(result, cuisine, on="CUISINECODE")

In [33]:
merge1.sort_values(by="ratio",ascending=False).ratio[0:20].mean()

73.712375989227397

In [34]:
merge1.head()

,CUISINECODE,VIOLCODE,CP,UNCP,ratio,number,CODEDESC
0,0,04A,0.115108,0.017416,6.609448,16,Not Listed/Not Applicable
1,0,10F,0.107914,0.124865,0.864244,15,Not Listed/Not Applicable
2,0,10B,0.071942,0.058900,1.221433,10,Not Listed/Not Applicable
3,0,0,0.057554,0.021083,2.729823,8,Not Listed/Not Applicable
4,0,04L,0.057554,0.072013,0.799221,8,Not Listed/Not Applicable


In [35]:
import csv

In [36]:
violation = pd.read_csv("data/nyc_inspection_data/Violation.txt",sep='","',quoting=csv.QUOTE_NONE)

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [37]:
violation['VIOLCODE'] = violation.VIOLATIONCODE

In [38]:
violation.head()

,"""STARTDATE",ENDDATE,CRITICALFLAG,VIOLATIONCODE,"VIOLATIONDESC""",VIOLCODE
0,"""1901-01-01 00:00:00",2003-03-23 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('...",01A
1,"""2003-03-24 00:00:00",2005-02-17 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('...",01A
2,"""2005-02-18 00:00:00",2007-06-30 00:00:00,Y,01A,"Current valid <a onmouseover=""ShowContent('...",01A
3,"""2007-07-01 00:00:00",2008-06-30 00:00:00,Y,01A,"Current valid permit, registration or other au...",01A
4,"""2008-07-01 00:00:00",2009-08-01 00:00:00,Y,01A,"Current valid permit, registration or other au...",01A


In [39]:
violation["ENDDATE"] = pd.to_datetime(violation.ENDDATE)

In [40]:
violation = violation[violation.ENDDATE>=pd.to_datetime("2014-01-01 00:00:00")]

In [41]:
violation.head()

,"""STARTDATE",ENDDATE,CRITICALFLAG,VIOLATIONCODE,"VIOLATIONDESC""",VIOLCODE
39,"""2010-07-26 00:00:00",2099-12-31,Y,02A,"Food not cooked to required minimum temperature.""",02A
46,"""2010-07-26 00:00:00",2099-12-31,Y,02B,"Hot food item not held at or above 140� F.""",02B
53,"""2010-07-26 00:00:00",2099-12-31,Y,02C,Hot food item that has been cooked and refrige...,02C
60,"""2010-07-26 00:00:00",2099-12-31,Y,02D,Precooked potentially hazardous food from comm...,02D
67,"""2010-07-26 00:00:00",2099-12-31,Y,02E,"Whole frozen poultry or poultry breasts, other...",02E


In [42]:
merge2 = pd.merge(merge1, violation,on="VIOLCODE")

In [43]:
merge2[merge2.VIOLATIONCODE=="10A"][merge2.CUISINECODE==14]

/Users/yutongpang/.virtualenvs/sqlVen27/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,CUISINECODE,VIOLCODE,CP,UNCP,ratio,number,CODEDESC,"""STARTDATE",ENDDATE,CRITICALFLAG,VIOLATIONCODE,"VIOLATIONDESC"""
800,14,10A,0.020676,0.011016,1.876848,315,Caf�/Coffee/Tea,"""2010-07-26 00:00:00",2099-12-31,N,10A,Toilet facility not maintained and provided wi...


In [44]:
merge2 = merge2[merge2.number>100]

In [45]:
merge2 = merge2.sort_values(by="ratio",ascending=False)

In [46]:
merge2.head()

,CUISINECODE,VIOLCODE,CP,UNCP,ratio,number,CODEDESC,"""STARTDATE",ENDDATE,CRITICALFLAG,VIOLATIONCODE,"VIOLATIONDESC"""
2107,49,04C,0.029013,0.008885,3.265526,541,Japanese,"""2010-07-26 00:00:00",2099-12-31,Y,04C,Food worker does not use proper utensil to eli...
734,14,20D,0.011487,0.003656,3.141482,175,Caf�/Coffee/Tea,"""2010-07-26 00:00:00",2099-12-31,N,20D,�Choking first aid� poster not posted. �Alcoho...
48,51,04A,0.053348,0.017416,3.063223,145,"Juice, Smoothies, Fruit Salads","""2010-07-26 00:00:00",2099-12-31,Y,04A,Food Protection Certificate not held by superv...
2017,29,10E,0.020158,0.006647,3.032475,130,Donuts,"""2010-07-26 00:00:00",2099-12-31,N,10E,Accurate thermometer not provided in refrigera...
40,43,04A,0.050991,0.017416,2.927868,193,"Ice Cream, Gelato, Yogurt, Ices","""2010-07-26 00:00:00",2099-12-31,Y,04A,Food Protection Certificate not held by superv...


In [48]:
merge2.ratio[0:20].mean()

2.3700921634985899

In [47]:
merge2.columns =["CUISINECODE","VIOLCODE","CP","UNCP","ratio","number","CODEDESC","STARTDATE","ENDDATE","CRITICALFLAG","VIOLATIONCODE","VIOLATIONDESC"]

In [49]:
ans = map(lambda x1,x2,x3,x4: ((x1,x2),x3,x4) ,merge2.CODEDESC, merge2.VIOLATIONDESC, merge2['ratio'], merge2.number)

In [52]:
ans = ans[0:20]

In [54]:
ans[1] =  (('Café/Coffee/Tea',
   '\xef\xbf\xbdChoking first aid\xef\xbf\xbd poster not posted. \xef\xbf\xbdAlcohol and pregnancy\xef\xbf\xbd warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted. Inspection report sign not posted."'),
  3.1414818233282684,
  175)

In [56]:
ans[-1] =  (('Café/Coffee/Tea',
   'Toilet facility not maintained and provided with toilet paper, waste receptacle and self-closing door."'),
  1.876847758271718,
  315)

In [57]:
import pickle

In [58]:
with open('vio.pickle', 'wb') as f:
    pickle.dump(ans[0:20], f)

In [118]:
unicode("Caf\xe9", "Latin-1") 

u'Caf\xe9'